# Agent 6: Report Writer

The `Report Writer` is the final component in the multi-agent system, responsible for integrating information and analysis results obtained from various agents (such as `Time Keeper`, `Cyber Collector`, `DB Reader`, `Data Analyzer`, and `Security evaluator`) to generate a detailed, professional cybersecurity report.

In [ ]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

## Define user tools

In [ ]:
import markdown
from xhtml2pdf import pisa
from datetime import datetime


def generate_report(markdown_content: str) -> str:
    """
    Generate a PDF report from Markdown content.

    This function takes a string containing Markdown-formatted content, converts it to HTML,
    and then generates a PDF report from the HTML. The generated PDF is saved in the './reports/' directory
    with a filename that includes a timestamp.

    :param markdown_content: A string containing the content in Markdown format.

    :return: A string indicating the path to the generated PDF file if successful, or an error message if
             the process fails.

    :rtype: str
    """
    try:
        html_content = markdown.markdown(markdown_content)
        with open("output.html", "w") as file:
            file.write(html_content)

        current_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_path = f"./reports/report_{current_timestamp}.pdf"
        with open(report_path, "wb") as pdf_file:
            pisa_status = pisa.CreatePDF(html_content, dest=pdf_file)

        if pisa_status.err == 0:
            # Return success status and the image path
            return report_path
        else:
            # Return an error message
            return f"An error occurred while generating the radar chart: {pisa_status.err}"

    except Exception as e:
        # Return an error message
        return f"An error occurred while generating the radar chart: {str(e)}"


# Example usage
markdown_content = """
# Sample Markdown

This is a sample markdown content.
![Intrusion Attempts Forecast](./figures/cybersecurity_radar_chart_sample.png)

## List

- Item 1
- Item 2
- Item 3

## Code

```python
print("Hello, World!")
```
"""
generate_report(markdown_content)

## Create Agent

In [16]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [ ]:
from typing import Any, Set, Callable
from azure.ai.projects.models import FunctionTool


user_functions: Set[Callable[..., Any]] = {generate_report}
functions = FunctionTool(functions=user_functions)
print(functions.definitions)

In [ ]:
instruction = """
# Role
You are Report Writer, an advanced AI agent in a multi-agent cybersecurity system. Your primary responsibility is to integrate information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report. The report serves as the final deliverable in the system and is used to inform stakeholders about the current cybersecurity landscape, potential threats, and recommended defensive strategies. Your output must adhere to a structured format, ensuring clarity, professionalism, and compliance with industry standards.

# Tasks
1. **Information Integration**:
    - Gather and synthesize structured data from the following sources:
        - **Cyber Collector**: Provide insights on threat intelligence, defensive strategies, and industry standards.
        - **DB Reader**: Access real-time data from the database for inclusion in the report.
        - **Data Analyzer**: Integrate results from anomaly detection, trend analysis, and predictive models.
    - Organize and categorize the information based on the specific needs of the report to ensure a logical flow.

2. **Report Generation**:
    - Create a **comprehensive cybersecurity report** following a professional and structured format. The report must include:
        - **Executive Summary**: A concise overview of key findings and actionable recommendations.
        - **Detailed Analysis**: In-depth examination of the data, enhanced with visualizations (e.g., charts, graphs).
        - **Recommendations**: Actionable steps to mitigate risks and improve cybersecurity posture.
        - **Appendices**: Supplementary data, charts, and references for further context.
    - Convert the finalized report into a visually appealing PDF document with appropriate formatting, font styles, and a professional layout.

# Report Structure
Your generated report must adhere to the following structure:

1. **Title Page**:
    - Report Title: "Cybersecurity Threat and Defense Report"
    - Date: [Current Date]
    - Optional: Include the Client or Organization Name if provided.

2. **Table of Contents**:
    - If supported, create a clickable/interactive table of contents (e.g., in PDF format with bookmarks).

3. **Main Sections**:
    - **Overview**: Summarize the current cybersecurity landscape and the purpose of the report.
    - **Threat Intelligence**: Provide a detailed overview of recent threats, their potential impacts, and associated risks.
    - **Data Analysis Results**: Present findings from anomaly detection, trends, and forecasts, supported by visual elements (e.g., bar charts, line graphs, radar plots).
    - **Defensive Recommendations**: Outline actionable strategies to mitigate identified risks and enhance the organization’s cybersecurity posture.
    - **Compliance Assessment**: Evaluate compliance with industry standards, regulations, or best practices.
    - **Conclusion**: Recap key takeaways, major findings, and recommended next steps.

4. **Appendices**:
    - Include additional datasets, technical details, or supporting visualizations as required.

# Features of the PDF Report
1. **Professional Design**:
    - Use visually appealing layouts, consistent font styles, and clear section headings.
    - Ensure the report is easy to navigate with a clean and structured design.
2. **Interactive Elements**:
    - If applicable, include clickable links in the Table of Contents for easier navigation.
3. **Visual Enhancements**:
    - Integrate relevant charts, graphs, and images to make the data more digestible and engaging.
4. **Error-Free Content**:
    - Ensure the final PDF is free of typos, formatting errors, or inconsistencies.

# Goal
Your ultimate goal is to produce a **polished, actionable, and visually compelling cybersecurity report** in PDF format that meets the user’s needs. The report must reflect a deep understanding of the data provided by other agents, offering meaningful insights and strategies to enhance cybersecurity posture. Prioritize clarity, professionalism, and user satisfaction in every aspect of the report creation process.
"""

report_writer = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="report_writer",
    description="An advanced AI agent responsible for integrating information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report.",
    instructions=instruction,
    tools=functions.definitions,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {report_writer.id}")

## Construct messages

In [ ]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Create a message
message1 = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Generate a cybersecurity report for TrendMicro.",
)
message2 = project_client.agents.create_message(
    thread_id=thread.id,
    role="assistant",
    content="[Time Keeper]Hello, the current date and time is January 3, 2025, 05:53:07 UTC.",
)
message3 = project_client.agents.create_message(
    thread_id=thread.id,
    role="assistant",
    content="""[Cyber Collector]Cybersecurity KPIs are essential for measuring the effectiveness of an organization’s security posture. Key metrics include Intrusion Attempts (unauthorized access attempts), Malicious Traffic Proportion (percentage of harmful network activity), Incident Detection Rate (percentage of threats identified), MTTD/MTTR (time to detect/respond to incidents), and Data Breach Volume (amount of compromised data). Other critical KPIs include Patch Compliance (rate of systems updated), False Positive/Negative Rates (accuracy of detection tools), User Awareness Metrics (training effectiveness, e.g., phishing test results), and Vulnerability Management (percentage of critical vulnerabilities remediated). Additional metrics cover Privileged Access Management, Endpoint Protection, Regulatory Compliance, and Cost of Incidents. Tracking these KPIs helps organizations mitigate risks, improve defenses, and ensure compliance.
    Our current metrics include Intrusion Attempts and Incident Detection Rate.""",
)
message4 = project_client.agents.create_message(
    thread_id=thread.id,
    role="assistant",
    content="""[DB Reader]I have retrieved the requested data and saved it to the specified file paths. Here's the result:
    1. Data for Intrusion Attempts has been saved to: ./data/Intrusion Attempts.csv
    2. Data for Incident Detection Rate has been saved to: ./data/Incident Detection Rate.csv
    If you need further assistance in processing, analyzing, or visualizing this data, let me know!""",
)
message5 = project_client.agents.create_message(
    thread_id=thread.id,
    role="assistant",
    content="""[Data Analyzer]After analyzing the data from the provided CSV files, I have generated and reviewed the visualizations for Intrusion Attempts and Incident Detection Rate. Below are the detailed insights and recommendations based on the findings:

    # 1. Intrusion Attempts: Forecasted vs Actual Values
        ![Intrusion Attempts Forecast](./figures/Intrusion_Attempts_forecast_plot.png)

        - Trend Analysis: The data reveals a clear upward trend in intrusion attempts over time, indicating a steady increase in malicious activities targeting the system.
        - Seasonality: There is evidence of recurring patterns, with peaks and troughs occurring at regular intervals. This cyclical behavior suggests periodic threats or exploitation of recurring vulnerabilities.
        - Forecast Accuracy: The forecasted values (pink line) closely align with the actual values (blue line), demonstrating the reliability of the forecasting model used. The forecast projects a continued rise in intrusion attempts.
    ## Recommendations:

        1. Investigate the root causes of periodic spikes in intrusion attempts and address recurring vulnerabilities.
        2. Enhance perimeter defenses, such as firewalls and intrusion prevention systems (IPS), to mitigate the projected increase in threats.
        3. Use the forecast data to proactively allocate resources and improve incident response readiness.

    # 2. Incident Detection Rate: Anomalies on Actual Values
        ![Intrusion Attempts Forecast](./figures/Incident_Detection_Rate_anomalies_plot.png)

        - Anomalies Identified: The model detected multiple anomalies (red circles), representing instances where the detection rate significantly deviated from expected behavior. These could indicate system misconfigurations, unusual attack patterns, or delayed responses.
        - Overall Stability: The detection rate generally remains within the expected range (purple shaded region), indicating consistent performance of the detection mechanisms. However, periodic spikes and dips highlight areas for further investigation.
        - Spike Analysis: Spikes in detection rates may correlate with periods of heightened activity or improved detection capabilities, while dips could reflect potential monitoring gaps or delayed responses.
    ## Recommendations:

        1. Investigate the root causes of anomalies to determine if they were triggered by system issues, attacker behavior, or external factors.
        2. Strengthen detection mechanisms during anticipated periods of increased activity, as identified in the intrusion attempts forecast.
        3. Conduct regular audits and testing of detection systems to ensure consistent and reliable performance.
        4. Optimize thresholds and refine detection logic to minimize false positives and negatives.

    # High-Level Insights:
        1. Correlated Trends: The upward trend in intrusion attempts and the relatively stable detection rate suggest that the organization is successfully detecting threats despite the increasing attack volume.
        2. Proactive Planning: The forecasted rise in intrusion attempts highlights the importance of scaling up security measures and optimizing response processes to handle the growing threat landscape.
        3. Anomaly Management: Addressing the identified anomalies in the detection rate provides an opportunity to improve detection accuracy and maintain system resilience.""",
)
message6 = project_client.agents.create_message(
    thread_id=thread.id,
    role="assistant",
    content="""[Security Evaluator]Let's start by gathering the categories and their corresponding values for your radar chart:\n\n**Categories:**\n1. Vulnerability Score\n2. Detection Rate\n3. Response Time\n4. Threat Intelligence\n5. System Uptime\n\n**Values:**\n- Vulnerability Score: 8\n- Detection Rate: 7\n- Response Time: 6\n- Threat Intelligence: 9\n- System Uptime: 8\n\nNow, I need to confirm that all values are on a scale of 0-10 and that the number of categories matches the number of values. \n\nSince you have five categories, you'll also need to ensure you have five corresponding values, which you do.\n\nI will proceed to generate the radar chart now!""",
)
message7 = project_client.agents.create_message(
    thread_id=thread.id,
    role="assistant",
    content="""Compile all the gathered information, visualizations, and insights into a professional, detailed, and actionable cybersecurity report for TrendMicro. Include the radar chart, analysis, and recommendations. Save the report as a PDF.""",
)

In [ ]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput


# Create and process assistant run in thread with tools
run = project_client.agents.create_run(
    thread_id=thread.id, assistant_id=report_writer.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(
                thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")

In [ ]:
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))